In [2]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

WORKING_DIR_AND_PYTHON_PATHS = os.path.join('/', *os.getcwd().split("/")[:-1])
# print(f'before {sys.path}')
sys.path.append(WORKING_DIR_AND_PYTHON_PATHS)
# print(f'after {sys.path}')

In [3]:
plt.style.use('seaborn')
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

In [1]:
WINDOW_SIZE = 20
FILE_PATH = '../dataset'

In [5]:
tot_baseball = pd.read_csv(os.path.join(FILE_PATH,'total_baseball.csv'))

In [6]:
def make_time_series(df,window_size):
    
    barell = ['2타','3타','홈런']
    on_base = ['안타','2타','3타','홈런','사구','볼넷','고4']
    out = ['병살','삼진','희타']
    etc = ['타수','도루','투구']
    is_bin = ['선발']


    # for calcaul
    total_base =['타수','볼넷','고4','사구','희타']
    hit = ['안타','2타','3타','홈런']
    
    sample_list = []
    
    for i in range(1,len(df),window_size):
        single = df[-i:-(i+window_size):-1].groupby(['이름','year'])[on_base+out+etc].sum()
        single['num_game'] = df[-i:-(i+window_size):-1].shape[0]
        single['on_base']= single[on_base].sum(axis=1)/single[total_base].sum(axis=1)
        single['slg'] = (single[hit] * [1,2,3,4]).sum(axis=1)/single['타수']
        sample_list.append(single)
    
    result = pd.concat(sample_list).reset_index()
    result.index = ['t'+str(i) for i in range(1,len(result)+1)]
    return result

In [6]:
base_2021=tot_baseball[(tot_baseball['year']==2021)]

In [8]:
base_2021.columns

Index(['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타',
       '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타',
       '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA', 'year'],
      dtype='object')

In [11]:
base_2021_ts = []

In [12]:
players= base_2021['이름'].value_counts().keys().to_list()
for player in players:
    sample_df = tot_baseball[(tot_baseball['year']==2021)&(tot_baseball['이름']==player)]
    base_2021_ts.append(make_time_series(sample_df,WINDOW_SIZE))

In [13]:
base_2021_ts = pd.concat(base_2021_ts)

In [14]:
base_2021_ts

,이름,year,안타,2타,3타,홈런,사구,볼넷,고4,병살,삼진,희타,타수,도루,투구,num_game,on_base,slg
t1,피렐라,2021,20,3,0,4,1,5,0,2,13,0,85,1,352,20,0.362637,0.494118
t2,피렐라,2021,30,4,0,4,2,9,1,0,11,0,80,3,336,20,0.543478,0.675000
t3,피렐라,2021,21,1,1,6,0,12,1,1,13,0,74,1,321,20,0.482759,0.675676
t4,피렐라,2021,23,5,0,4,1,10,0,0,14,0,83,2,380,20,0.457447,0.590361
t5,피렐라,2021,25,4,1,5,1,3,0,1,8,0,78,2,316,19,0.475610,0.717949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t1,민병헌,2021,8,3,0,0,0,6,0,2,11,1,42,0,183,14,0.346939,0.333333
t1,이해창,2021,9,3,0,0,1,1,0,0,9,2,39,0,166,14,0.325581,0.384615
t1,이진영,2021,8,2,1,2,4,3,0,0,19,0,35,1,189,13,0.476190,0.657143
t1,김수환,2021,9,2,0,2,0,2,0,2,17,0,41,0,172,13,0.348837,0.512195


In [25]:
base_2021_ts.to_csv('base_ts_2021.csv',index_label='Index_ts')

In [28]:
temp = pd.read_csv('base_ts_2021.csv').set_index('Index_ts')

In [29]:
temp

,이름,year,안타,2타,3타,홈런,사구,볼넷,고4,병살,삼진,희타,타수,도루,투구,num_game,on_base,slg
Index_ts,,,,,,,,,,,,,,,,,,
t1,피렐라,2021,20,3,0,4,1,5,0,2,13,0,85,1,352,20,0.362637,0.494118
t2,피렐라,2021,30,4,0,4,2,9,1,0,11,0,80,3,336,20,0.543478,0.675000
t3,피렐라,2021,21,1,1,6,0,12,1,1,13,0,74,1,321,20,0.482759,0.675676
t4,피렐라,2021,23,5,0,4,1,10,0,0,14,0,83,2,380,20,0.457447,0.590361
t5,피렐라,2021,25,4,1,5,1,3,0,1,8,0,78,2,316,19,0.475610,0.717949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t1,민병헌,2021,8,3,0,0,0,6,0,2,11,1,42,0,183,14,0.346939,0.333333
t1,이해창,2021,9,3,0,0,1,1,0,0,9,2,39,0,166,14,0.325581,0.384615
t1,이진영,2021,8,2,1,2,4,3,0,0,19,0,35,1,189,13,0.476190,0.657143
